In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import KeyedVectors, Word2Vec

SPECIAL_CHARS = '[^A-Za-z0-9 ]+'

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hsichengyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hsichengyun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def preprocess(text):
    tokenized = [word for sent in [re.sub(SPECIAL_CHARS, '', element).split(' ') for 
                                   element in nltk.sent_tokenize(text)] for word in sent]
    lowered = [word.lower() for word in tokenized]
    return lowered

In [3]:
dataset = pd.read_csv('dataset.csv')

In [4]:
ls = list()
for ix in range(dataset.shape[0]):
    content = list()
    for e in dataset.iloc[ix,4:9]:
        content.append(str(e))
    s = ','.join(content)
    ls.append(s)

In [5]:
dataset['everything'] = ls

In [6]:
description_texts = [preprocess(text) for text in dataset['everything'].tolist()]

In [7]:
# # Take out the stopwords
# for i in range(len(description_texts)):
#     description_texts[i] = [word for word in description_texts[i] if word not in stopwords.words('english')]

# I use movie title as unique key. So I map out the title and the tokenised sentences

title_text = dict(zip(dataset['title'].tolist(), description_texts))

In [9]:
model = Word2Vec(min_count=20,
                window=2,
                size=300,
                sample=6e-5,
                alpha=0.03,
                min_alpha=0.0007,
                negative=20,
                )

In [ ]:
def get_vectors(first_map, second_map):
    first_vec  = dict()
    for uid, content in first_map.items():
        temp = list()
        for element in content:
            try:
                temp.append(second_map[element])
            except KeyError:
                pass
        first_vec[uid] = np.mean(temp, axis=0)
    
    return first_vec


def get_most_similar(lookup_id):

    sim = list()
    
    lookup_map = title_vec
    subject_map = title_vec
#     else:
#         raise ValueError('Invalid value for parameter kind.')
        
    for uid, vec in lookup_map.items():
        thisSim = cosine_similarity(vec.reshape(1, -1), subject_map[lookup_id].reshape(1, -1))
        sim.append((uid, thisSim[0][0]))

    return sorted(sim, key=lambda x: x[1], reverse=True)

def top_10_similar(title):
    
    x = get_most_similar(title)[1:11]
#     for e in x:
# #         rating = dataset.loc[dataset['title']==e[0]]['rating_value'].values.tolist()[0]
# #         votes = dataset.loc[dataset['title']==e[0]]['votes'].values.tolist()[0]
# #         print(f"Movie title: {e[0]}\nScores: {rating}\nVotes: {votes}\nSimilarity: {e[1]}\n")
#         genres = test.loc[test['title']==e[0]['genres'].values.tolist()]
#         print(f"Movie title: {e[0]}\nGenres: {genres}\nSimilarity: {e[1]}\n")
    print(x)

In [ ]:
title_vec = get_vectors(title_text, model)

In [ ]:
top_10_similar('House of Cards')

# Test 6k dataset

In [ ]:
data6k = pd.read_csv('netflixMovieDb.csv')
data6k = data6k.append(pd.read_csv('netflixTvshowDb.csv')).append(pd.read_csv('dMoviesDb.csv')).append(pd.read_csv('dTvshowsDb.csv'))

In [ ]:
data6k.shape

In [ ]:
ls = list()
for ix in range(data6k.shape[0]):
    content = list()
    for e in data6k.iloc[ix,4:9]:
        content.append(str(e))
    s = ','.join(content)
    ls.append(s)
    
data6k['everything'] = ls

In [ ]:
description_texts = [preprocess(text) for text in data6k['everything'].tolist()]

In [ ]:
# Take out the stopwords
for i in range(len(description_texts)):
    description_texts[i] = [word for word in description_texts[i] if word not in stopwords.words('english')]

# I use movie title as unique key. So I map out the title and the tokenised sentences

title_text = dict(zip(data6k['title'].tolist(), description_texts))

In [ ]:
model = Word2Vec(description_texts, min_count=2)

In [ ]:
title_vec = get_vectors(title_text, model)

In [ ]:
top_10_similar('Ultraman')

In [ ]:
top_10_similar('House of Cards')

Spacy

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import pandas as pd

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
data6k.iloc[0, 5]

In [ ]:
doc = nlp("Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.")

In [ ]:
displacy.render(doc)

In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
displacy.serve(doc, style='ent')

In [ ]:
nTv = pd.read_csv('dataset.csv')

In [ ]:
tv_li = nTv.loc[nTv['type']=='tvshow'].title.tolist()

In [ ]:
tv_li

In [ ]:
doc = nTv.loc[nTv['title']=='The Witcher'].description.values.tolist()[0]
doc

In [ ]:
doc = nlp(doc)

In [ ]:
for token in doc:
    print(token.text)